In [1]:
!pip install gym


You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:

import tensorflow as tf
#tf.enable_eager_execution()


In [3]:

from mcts import Node, run_mcts
from gamecomponents import Policy
from game import Game
from replaybuffer import ReplayBuffer
from network import Network, SharedStorage
import random
import numpy as np
from helpers import KnownBounds
from muzero_core import play_game, train_network, MuZeroConfig, make_aigym_config

In [4]:
c = make_aigym_config('CartPole-v0')
s = SharedStorage()
r = ReplayBuffer()

    
c.num_simulations = 60
c.training_steps = 150
c.batch_size = 32
c.td_steps = 25
c.discount = 1. 
c.lr_init = 0.02
c.lr_decay_steps = 100e3
c.max_moves = 2000

# a little bit of help on the known bounds.
c.known_bounds = KnownBounds(max = 20, min = -20)
c.epsilon = 0.001

c.num_unroll_steps = 4 # As we are moving in 8-action steps, this means we discard some information, but turns out to work better than 8

Network.N = 6
Network.grad_scale = (1.,1.,1.)

for _ in range(30):
  n = s.latest_network()
  g1 = play_game(c, n)
  r.save_game(g1)
  print(sum(g1.rewards))

19.0
46.0
25.0
20.0
33.0
34.0
9.0
12.0
10.0
23.0
24.0
11.0
23.0
13.0
37.0
38.0
20.0
16.0
13.0
16.0
42.0
24.0
22.0
44.0
10.0
19.0
20.0
8.0
19.0
14.0


In [ ]:


from statistics import mean 

N = 10
st = 0
for inter in range(400):

  train_network(c,s,r)

  n = s.latest_network()

  tot = 0
  N = 0
  better = True
  while better:
    g1 = play_game(c, n)
    print(g1.length())

    tot += (g1.length())
    N += 1
    better = tot/N > max(r.game_len)*0.5 and N < 25
    r.save_game(g1)
    
  print((inter, tot/N))

  print('----')






COMET INFO: Experiment is live on comet.ml https://www.comet.ml/pnorridge/muzero-cartpolenstep/5a6c83aea0704b8f9ff552d35811d720



Instructions for updating:
Use tf.identity instead.
(0, (12.796543, 12.737388, 1.3862944))
(100, (2.6498737, 7.0753303, 1.3858738))
60
[16.340406]
16
[16.306362]
11
[16.260435]
(0, 29.0)
----
(0, (3.6446798, 7.075293, 1.3845377))
(100, (2.4362943, 7.07529, 1.3856039))
12
[17.32642]
(1, 12.0)
----
(0, (2.5578785, 7.0752883, 1.3877115))
(100, (2.7554982, 7.075288, 1.387814))
56
[17.777418]
29
[17.777472]
13
[17.777435]
16
[17.768341]
(2, 28.5)
----
(0, (2.689096, 7.075289, 1.3864532))
(100, (2.3906844, 7.075288, 1.3867577))
18
[16.477818]
(3, 18.0)
----
(0, (2.4781284, 7.0752873, 1.3864026))
(100, (2.2542973, 7.075289, 1.3853787))
35
[17.708488]
17
[17.683338]
(4, 26.0)
----
(0, (2.565545, 7.075289, 1.3871074))
(100, (2.3738766, 7.0752916, 1.3851656))
41
[16.856922]
15
[16.856495]
(5, 28.0)
----
(0, (2.5244129, 7.075295, 1.3857129))
(100, (2.9853764, 7.075298, 1.3869172))
11
[17.67877]
(6, 11.0)
----
(0, (2.3591466, 7.075299, 1.3888488))
(100, (2.9110422, 7.0753136, 1.3847994))
19
[18.30